In [13]:
from datasets import load_dataset
from datasets import load_metric
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering
from functools import partial
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from datasets import load_metric
import torch
from helper import collate_fn, get_train_features, get_validation_features, post_process_predictions, predict, val_collate_fn
from lab6_train import *

lr = 2e-5
n_epochs = 3
weight_decay = 0.01
warmup_steps = 200
bert_map = {
    'bengali': 'xlm-roberta-base', 
    'english': 'xlm-roberta-base', 
    'indonesian': 'xlm-roberta-base', 
    'arabic': 'xlm-roberta-base'
}
device = 'cuda'

compute_squad = load_metric("squad_v2")
dataset = load_dataset("copenlu/answerable_tydiqa")

for split in dataset.keys():
    dataset[split] = dataset[split].add_column('id', list(range(len(dataset[split]))))

for language, bert in list(bert_map.items()):
    print(f'Language: {language}')
    language_dataset = dataset.filter(lambda example: example['language'] == language)
    tk = AutoTokenizer.from_pretrained(bert, max_len=300)

    tokenized_train_dataset = language_dataset['train'].map(partial(get_train_features, tk), batched=True, remove_columns=language_dataset['train'].column_names)
    tokenized_validation_dataset = language_dataset['validation'].map(partial(get_validation_features, tk), batched=True, remove_columns=language_dataset['validation'].column_names)

    train_dl = DataLoader(tokenized_train_dataset, collate_fn=collate_fn, shuffle=True, batch_size=4)

    model = AutoModelForQuestionAnswering.from_pretrained(bert).to(device)
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        'weight_decay': weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(
            nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

    optimizer = AdamW(optimizer_grouped_parameters, lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        warmup_steps,
        n_epochs * len(train_dl)
    )

    losses = train(
        model,
        train_dl,
        optimizer,
        scheduler,
        n_epochs,
        device
    )
    torch.save(model, f'{language}_xlm-roberta-base_span_detection.pt')
    model = torch.load(f'{language}_xlm-roberta-base_span_detection.pt')
    for language, bert in list(bert_map.items()):
        language_dataset = dataset.filter(lambda example: example['language'] == language)
        tk = AutoTokenizer.from_pretrained(bert, max_len=300)

        tokenized_train_dataset = language_dataset['train'].map(partial(get_train_features, tk), batched=True, remove_columns=language_dataset['train'].column_names)
        tokenized_validation_dataset = language_dataset['validation'].map(partial(get_validation_features, tk), batched=True, remove_columns=language_dataset['validation'].column_names)

        train_dl = DataLoader(tokenized_train_dataset, collate_fn=collate_fn, shuffle=True, batch_size=4)
        val_dl = DataLoader(tokenized_validation_dataset, collate_fn=val_collate_fn, batch_size=32)
        
        logits = predict(model, val_dl, device)
        predictions = post_process_predictions(language_dataset['validation'], tokenized_validation_dataset, logits)
        formatted_predictions = [{'id': k, 'prediction_text': v, 'no_answer_probability': 0.} for k, v in predictions.items()]
        gold = [{
            'id': example['id'],
            'answers': {
                'text': example['annotations']['answer_text'],
                'answer_start': example['annotations']['answer_start']}
            }
            for example in language_dataset['validation']]


        print(compute_squad.compute(references=gold, predictions=formatted_predictions))


Language: bengali


Map:   0%|          | 0/4779 [00:00<?, ? examples/s]

Map:   0%|          | 0/224 [00:00<?, ? examples/s]

Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 224/224 [00:00<00:00, 1121.04it/s]


{'exact': 28.125, 'f1': 34.021687637759065, 'total': 224, 'HasAns_exact': 28.125, 'HasAns_f1': 34.021687637759065, 'HasAns_total': 224, 'best_exact': 28.125, 'best_exact_thresh': 0.0, 'best_f1': 34.021687637759065, 'best_f1_thresh': 0.0}
Language: english


...

{'exact': 34.17296389588581, 'f1': 39.967987795222456, 'total': 1191, 'HasAns_exact': 34.17296389588581, 'HasAns_f1': 39.967987795222456, 'HasAns_total': 1191, 'best_exact': 34.17296389588581, 'best_exact_thresh': 0.0, 'best_f1': 39.967987795222456, 'best_f1_thresh': 0.0}
Language: arabic


Map:   0%|          | 0/29598 [00:00<?, ? examples/s]

Map:   0%|          | 0/1902 [00:00<?, ? examples/s]

Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 1902/1902 [00:01<00:00, 1219.61it/s]


{'exact': 30.336487907465827, 'f1': 38.44675862942356, 'total': 1902, 'HasAns_exact': 30.336487907465827, 'HasAns_f1': 38.44675862942356, 'HasAns_total': 1902, 'best_exact': 30.336487907465827, 'best_exact_thresh': 0.0, 'best_f1': 38.44675862942356, 'best_f1_thresh': 0.0}


In [17]:
metrics = {l: {} for l in bert_map.keys()}
for language, bert in list(bert_map.items()):
    print(f'Language: {language}')
    model = torch.load(f'{language}_xlm-roberta-base_span_detection.pt')
    for language2, bert in list(bert_map.items()):
        language_dataset = dataset.filter(lambda example: example['language'] == language2)
        tk = AutoTokenizer.from_pretrained(bert, max_len=300)

        tokenized_train_dataset = language_dataset['train'].map(partial(get_train_features, tk), batched=True, remove_columns=language_dataset['train'].column_names)
        tokenized_validation_dataset = language_dataset['validation'].map(partial(get_validation_features, tk), batched=True, remove_columns=language_dataset['validation'].column_names)

        train_dl = DataLoader(tokenized_train_dataset, collate_fn=collate_fn, shuffle=True, batch_size=4)
        val_dl = DataLoader(tokenized_validation_dataset, collate_fn=val_collate_fn, batch_size=32)
        
        logits = predict(model, val_dl, device)
        predictions = post_process_predictions(language_dataset['validation'], tokenized_validation_dataset, logits)
        formatted_predictions = [{'id': k, 'prediction_text': v, 'no_answer_probability': 0.} for k, v in predictions.items()]
        gold = [{
            'id': example['id'],
            'answers': {
                'text': example['annotations']['answer_text'],
                'answer_start': example['annotations']['answer_start']}
            }
            for example in language_dataset['validation']]


        metrics[language][language2] = compute_squad.compute(references=gold, predictions=formatted_predictions)

Language: bengali


100%|██████████| 1902/1902 [00:01<00:00, 1184.38it/s]


Language: english


100%|██████████| 1902/1902 [00:01<00:00, 1232.29it/s]


Language: indonesian


100%|██████████| 1902/1902 [00:01<00:00, 1224.58it/s]


Language: arabic


100%|██████████| 1902/1902 [00:01<00:00, 1226.49it/s]


In [27]:
# Pretty print metrics
print("Exact match scores, xlm-r span detection:")
print(' & '.join(['tuned lan'] + list(bert_map.keys())) + '\\\\')
for language in bert_map.keys():
    print(' & '.join([language] + [f'{metrics[language][language2]["exact"]:.2f}' for language2 in bert_map.keys()]) + '\\\\')
print(' & '.join(['Average'] + [f'{sum([metrics[language][language2]["exact"] for language2 in bert_map.keys()]) / len(bert_map.keys()):.2f}' for language in bert_map.keys()]) + '\\\\')



Exact match scores, xlm-r span detection:
tuned lan & bengali & english & indonesian & arabic\\
bengali & 28.12 & 21.31 & 20.49 & 13.72\\
english & 15.62 & 25.96 & 27.54 & 20.08\\
indonesian & 17.86 & 26.16 & 34.17 & 25.13\\
arabic & 18.30 & 26.16 & 29.89 & 30.34\\
Average & 20.91 & 22.30 & 25.83 & 26.17\\
